# Gaze Estimation

### 1. Generating measureble data 

In [ ]:
meter aqui um demo e depois (como é só demo), mostrar um exemplo de resultados e criar a o yes.txt

In [12]:
#Importing packages:
import cv2 #image manipulation
import sys #manipulating function arguments
import numpy as np #eventual mathemathic operations
from pydarknet import Detector, Image #openpose
import os #operating system interactions
dir_path = os.path.dirname(os.path.realpath(__file__))
from openpose import pyopenpose as op #open pose

ModuleNotFoundError: No module named 'cv2'

In [21]:
# coords2Quadrante calculates to which quadrant (out of 16) a tuple of coordinates refers
# the function's parameters include the coordinates - a tuple - and the image object
# It outputs the quadrant and the image with the calculated quadrant overlined
def coords2Quadrante(coords,img):
    overlay = img.copy() # safe copy of the image to apply alpha
    h, w = img.shape[:2] # gets the image shape (height, width) 
    h = int(h) # converts parameters to integers 
    w= int(w)
    
    # next, it compares the x and the y to a reference from the original image (x/2,x/4 or y/2,y/4)
    # note that the image was devided in 16 rectangles: firstly divided in four and each of these, devided
    # in four again
    
    #____________________________# 0
    # 0  \  1   \   4   \   5    #
    #-----------\----------------# h/4
    # 2  \  3   \   6   \   7    #
    #___________\________________# h/2
    # 8  \  9   \   12  \   13   #
    # ----------\----------------# (3h)/4
    # 10 \  11  \   13  \   15   #
    # __________\________________#  h
    # 0  w/4    w/2    3w/4    w


    if coords[0]<int(w/2):
        if coords[1]<int(h/2):
            if coords[0]<int(w/4):
                if coords[1]<int(h/4):
                    quadrante=0
                    cv2.rectangle(overlay, (0,0), (int(w/4),int(h/4)), (229,88,191), thickness=-1) 
                else:
                    quadrante=2
                    cv2.rectangle(overlay, (0,int(h/4)), (int(w/4),int(h/2)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int(h/4):
                    quadrante=1
                    cv2.rectangle(overlay, (int(w/4),0), (int(w/2),int(h/4)), (229,88,191), thickness=-1)
                 else:
                    quadrante=3               
                    cv2.rectangle(overlay, (int(w/4),int(h/4)), (int(w/2),int(h/2)), (229,88,191), thickness=-1)
        else:
            if coords[0]<int(w/4): 
                if coords[1]<int((3*h)/4):
                    quadrante=8
                    cv2.rectangle(overlay, (0,int(h/2)), (int(w/4),int((3*h)/4)), (229,88,191), thickness=-1)
                    
                else:
                    quadrante=10
                    cv2.rectangle(overlay, (0,int((3*h)/4)), (int(w/4),int(h)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int((3*h)/4):
                    quadrante=9
                    cv2.rectangle(overlay, (int(w/4),int(h/2)), (int(w/2),int((3*h)/4)), (229,88,191), thickness=-1)
                 else:
                    quadrante=11               
                    cv2.rectangle(overlay, (int(w/4),int((3*h)/4)), (int(w/2),h), (229,88,191), thickness=-1)
    else:
        if coords[1]<int(h/2): 
            if coords[0]<int((3*w)/4):
                if coords[1]<int(h/4):
                    quadrante=4
                    cv2.rectangle(overlay, (int(w/2),0), (int((3*w)/4),int(h/4)), (229,88,191), thickness=-1)
                 
                else:
                    quadrante=6
                    cv2.rectangle(overlay, ((int(w/2),int(h/4))), (int((3*w)/4),int(h/2)), (229,88,191), thickness=-1)
                   
            else:
                 if coords[1]<int(h/4):
                    quadrante=5
                    cv2.rectangle(overlay, (int((3*w)/4),0), (int(w),int(h/4)), (229,88,191), thickness=-1)
                    
                 else:
                    quadrante=7          
                    cv2.rectangle(overlay, (int((3*w)/4),int(h/4)), (int(w),int(h/2)), (229,88,191), thickness=-1)
                    
        else:
            if coords[0]<int((3*w)/4): 
                if coords[1]<int((3*h)/4):
                    quadrante=12
                    cv2.rectangle(overlay, (int(w/2),int(h/2)), (int((3*w)/4),int((3*h)/4)), (229,88,191), thickness=-1)
                else:
                    quadrante=14
                    cv2.rectangle(overlay, (int(w/2),int((3*h)/4)), (int((3*w)/4),int(h)), (229,88,191), thickness=-1)
            else:
                 if coords[1]<int((3*h)/4):
                    quadrante=13
                 else:
                    quadrante=15               
                    cv2.rectangle(overlay, (int((3*w)/4),int((3*h)/4)), (int(w),int(h)), (229,88,191), thickness=-1)                    
            
    new = cv2.addWeighted(overlay, 0.4, img, 1 - 0.4, 0, img) #purple overlay 
    return quadrante,new #returns the quadrant and a new picture, with the underlined quadrant

In [23]:
coords2Quadrante((924,1234),cv2.imread('image.png'))

NameError: name 'cv2' is not defined

In [14]:
# contextual information - classifying an object in long or short range interaction
def short_long(objeto):
    range='range'
    if objeto in ['tv','pottedplant','vase','cat']:
        range='long'
    if objeto in ['chair','dining table','refrigerator','microwave','sink','bowl','apple','banana','laptop',\
                'keyboard','mouse','knife','fork','backpack']:
        range='short'
    return range

In [15]:
short_long('tv')

'long'

### 2. Separating training-set and testing-set 

In [5]:
import random
import shutil
import os

In [6]:
def separaPlus():   
    dir="lixo"
    list = os.listdir(dir)
    j=False
    i=0
    
    for i in range(8):
        file=random.choice(list)
        while j==False:
            if file.endswith('.txt'):
                j=True
            else:
                 file=random.choice(list)
        srcpath0=os.path.join(dir, file)
        shutil.move(srcpath0, "treino")
        shutil.move(srcpath0[:-4]+".png","treino")
        list = os.listdir(dir)
        j=False
        
    for i in range(3):
        file=random.choice(list)
        while j==False:
            if file.endswith('.txt'):
                j=True
            else:
                file=random.choice(list)
        srcpath1=os.path.join(dir, file)
        shutil.move(srcpath1, "teste")
        shutil.move(srcpath1[:-4]+".png","teste")
        list = os.listdir(dir)
        j=False
separaPlus()

FileNotFoundError: [WinError 3] O sistema não conseguiu localizar o caminho especificado: 'lixo'

### 3. Creating the model

In [8]:
from sklearn.svm import SVC 
import numpy as np
import ast
from joblib import dump, load
from sklearn import svm, grid_search

In [9]:
with open("treino.txt") as file:
    lines = [line.strip() for line in file]
y = []
X = []

counter = 0
zeta = len(lines)
while counter != zeta:
    omega = ast.literal_eval(lines[counter])
    X += [omega[:10]]
    if omega[-1] == -1:
        y+=[0]
    else:
        y+=[omega[-1]]
    counter+=1

def svc_param_selection(X, y, nfolds):
    Cs = [1000,100]
    gammas = [1e-06]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search2 = grid_search.GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search2.fit(X, y)
    grid_search2.best_params_
    return grid_search2.best_params_
    
best = svc_param_selection(X, y, 3)

clf = SVC(C= best['C'],gamma=best['gamma'])
clf.fit(X, y)     

dump(clf, 'classificadorCALEIDO2.joblib', protocol=2)

FileNotFoundError: [Errno 2] No such file or directory: 'treino.txt'

###  4. Testing the model

In [10]:
with open("treino.txt") as file:
    lines2 = [line.strip() for line in file]
y2 = []
X2 = []
Ypredicted= []
counter2 = 0
good = 0
zeta2 = len(lines2)
confMatrix = np.zeros((16,16))

while counter2 != zeta2:
    omega2 = ast.literal_eval(lines2[counter2])
    X2 += [omega2[:10]]
    if omega2[-1] == -1:
        y2+=[0]
    else:
        y2+=[omega2[-1]]
    Ypredicted+= [str(clf2.predict(np.array([X2[counter2]]))).strip('['+']')]
    y2[counter2]= str(y2[counter2])
    
    confMatrix[int(Ypredicted[counter2])][int(y2[counter2])] += 1 
 
    
    
    if Ypredicted[counter2] == y2[counter2]:
        good+=1
    counter2+=1
persucess = (float(good)/float(zeta2)) * 100    
print('A percentagem de sucesso foi de: ' + str(persucess)+ ' %')
print(str(confMatrix))

def sum1(input):
    sum = 0
    for row in range (len(input)):
        for col in range(len(input[0])):
            sum = sum + input[row][col]

    return sum

FileNotFoundError: [Errno 2] No such file or directory: 'treino.txt'

In [11]:
print(sum1(confMatrix))

NameError: name 'sum1' is not defined